In [21]:
import netCDF4 as nc
import numpy as np
import rasterio
import shapely
from shapely.geometry import Polygon,shape
import json

# 读取NetCDF文件
nc_file = nc.Dataset('h3_data.nc', 'r')

# 获取六边形网格的坐标和值
hex_coords = nc_file.variables['hex'][:]
hex_values = nc_file.variables['公交设施服务'][:]


In [30]:
# 定义栅格数据的大小和范围
width = 1
height = 1
left, bottom, right, top = [0, 0, 10, 10]
transform = rasterio.transform.from_bounds(
    left, bottom, right, top, width, height)

# 定义六边形半径
radius = 0.5

# 将六边形网格转换为栅格数据
hex_raster = np.zeros((height, width))
for i, hex_str in enumerate(hex_coords):
    # 将六边形的字符串转换为 Polygon 对象
    polygon = shape(json.loads(hex_str))

    # 计算六边形在栅格数据中的位置
    col_offset = int(
        np.round((polygon.centroid.x - left) / (right - left) * width))
    row_offset = int(
        np.round((top - polygon.centroid.y) / (top - bottom) * height))

    # 创建六边形
    hexagon = shapely.geometry.Polygon([(polygon.centroid.x + np.cos(np.pi/3*i) * radius,
                                         polygon.centroid.y + np.sin(np.pi/3*i) * radius) for i in range(6)])

    # 将六边形转换为栅格数据
    hex_raster[row_offset, col_offset] = hex_values[i]
    hex_mask = rasterio.features.geometry_mask(
        [hexagon], out_shape=hex_raster.shape, transform=transform, invert=True)
    hex_raster[hex_mask] = np.nan

# 保存为 GeoTIFF 文件
with rasterio.open('output.tif', 'w', driver='GTiff', width=width, height=height, count=1,
                   dtype=hex_raster.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(hex_raster, 1)


IndexError: index -11 is out of bounds for axis 0 with size 1

In [27]:
polygon.centroid.x


32.526368526634656

In [15]:
str = '{"coordinates": [[[32.521098, 118.997583], [32.53231, 118.995159], [32.537581, 118.982805], [32.531639, 118.972876], [32.520426, 118.975302], [32.515156, 118.987656]]], "type": "Polygon"}'


In [20]:
from shapely.geometry import shape
import json
type(shape(json.loads(str)))

shapely.geometry.polygon.Polygon